In [45]:
import pandas as pd

In [46]:
import matplotlib.pyplot as plt

In [47]:
%matplotlib inline

In [139]:
df = pd.read_csv("data/season_1/features/51.csv")

In [49]:
df.dtypes

start_district_num                                                                              int64
time_slice                                                                                      int64
date                                                                                           object
order_count                                                                                   float64
succeed_count                                                                                 float64
gap                                                                                           float64
weekday                                                                                         int64
is_workday                                                                                      int64
order_30min                                                                                   float64
succeed_30min                                                                     

In [50]:
df["date"] = df["date"].apply(lambda x: pd.to_datetime(x, errors='coerce'))

In [140]:
df.dtypes

start_district_num                                                                              int64
time_slice                                                                                      int64
date                                                                                           object
order_count                                                                                   float64
succeed_count                                                                                 float64
gap                                                                                           float64
weekday                                                                                         int64
is_workday                                                                                      int64
order_30min                                                                                   float64
succeed_30min                                                                     

In [52]:
df

,start_district_num,time_slice,date,order_count,succeed_count,gap,weekday,is_workday,order_30min,succeed_30min,gap_30min,"('order_20min', 'succeed_20min', 'gap_20min', 'order_10min', 'succeed_10min', 'gap_10min')",order_20min,succeed_20min,gap_20min,order_10min,succeed_10min,gap_10min
0,1,1,2016-01-02,88.0,85.0,3.0,5,0,115.0,111.0,4.0,0,90,89,1,84,82,2
1,1,2,2016-01-02,97.0,90.0,7.0,5,0,90.0,89.0,1.0,0,84,82,2,88,85,3
2,1,3,2016-01-02,75.0,70.0,5.0,5,0,84.0,82.0,2.0,0,88,85,3,97,90,7
3,1,4,2016-01-02,60.0,58.0,2.0,5,0,88.0,85.0,3.0,0,97,90,7,75,70,5
4,1,5,2016-01-02,72.0,65.0,7.0,5,0,97.0,90.0,7.0,0,75,70,5,60,58,2
5,1,6,2016-01-02,86.0,68.0,18.0,5,0,75.0,70.0,5.0,0,60,58,2,72,65,7
6,1,7,2016-01-02,59.0,54.0,5.0,5,0,60.0,58.0,2.0,0,72,65,7,86,68,18
7,1,8,2016-01-02,44.0,42.0,2.0,5,0,72.0,65.0,7.0,0,86,68,18,59,54,5
8,1,9,2016-01-02,42.0,41.0,1.0,5,0,86.0,68.0,18.0,0,59,54,5,44,42,2
9,1,10,2016-01-02,38.0,34.0,4.0,5,0,59.0,54.0,5.0,0,44,42,2,42,41,1


In [149]:
from sklearn.metrics.scorer import make_scorer
import math

def MAPE_scorer(y, y_pred):
    error = 0
    num = len(y)
    for i in range(0, num):
        if y[i] > 0:
            error += math.fabs(y_pred[i] - y[i]) / y[i]
    #print('error, num:', error, num)
    if num > 0:
        return error / num
    else:
        return 0

my_scorer = make_scorer(MAPE_scorer)

In [10]:
y = df['gap'][:21*66*144]
MAPE_scorer(y, [1 for i in range(0, 21*66*144)])

error, num: 53370.3598616 199584


0.26740800796433484

In [141]:
# split training, CV, test set

training = df.loc[(df.date < '2016-01-17') | (df.date == '2016-01-18')]

training_time = training.loc[df.time_slice.isin([46, 58, 70, 82, 94, 106, 118, 130, 142])]

cv = df.loc[df['date'].isin(['2016-01-17','2016-01-19','2016-01-20',
                             '2016-01-21'])]

cv_time = cv.loc[df.time_slice.isin([46, 58, 70, 82, 94, 106, 118, 130, 142])]
# only catch time slice in test set

test = df.loc[(df['date'].isin(['2016-01-23','2016-01-27','2016-01-31']) &\
         df['time_slice'].isin([46, 58, 70, 82, 94, 106, 118, 130, 142]))
      |\
       (df['date'].isin(['2016-01-25','2016-01-29']) &\
         df['time_slice'].isin([58, 70, 82, 94, 106, 118, 130, 142]))]

In [126]:

import matplotlib
matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)

In [156]:
from sklearn import linear_model

for time in [46, 58, 70, 82, 94, 106, 118, 130, 142]:
    training_ts = training_time.loc[training_time.time_slice == time]
    cv_ts = cv_time.loc[cv.time_slice == time]
    test_ts = test.loc[test.time_slice == time]
    X_training = training_ts.as_matrix(columns=[
                                  'is_workday',
                                  'order_30min','succeed_30min',
                                  'order_20min', 'succeed_20min', 
                                  'order_10min', 'succeed_10min'])
    y_training = training_ts.as_matrix(columns=['gap']).ravel()
    X_cv = cv_ts.as_matrix(columns=[
                                  'is_workday',
                                  'order_30min','succeed_30min',
                                  'order_20min', 'succeed_20min', 
                                  'order_10min', 'succeed_10min'])
    y_cv = cv_ts.as_matrix(columns=['gap']).ravel()
    X_test = test_ts.as_matrix(columns=[
                                  'is_workday',
                                  'order_30min','succeed_30min',
                                  'order_20min', 'succeed_20min', 
                                  'order_10min', 'succeed_10min'])
    clf = linear_model.LinearRegression()
    clf.fit(X_training, y_training)
    #print(time, clf.score(X_training, y_training), clf.score(X_cv, y_cv),
    #     (MAPE_scorer(y_cv, clf.predict(X_cv))))
    #print(clf.coef_ ,clf.predict(X_test))
    print('time_slice {}: train {}, cv {}'.format(time, MAPE_scorer(y_training, clf.predict(X_training)),
         MAPE_scorer(y_cv, clf.predict(X_cv))))

time_slice 46: train 1.36674685667887, cv 0.5331958382649765
time_slice 58: train 0.4470249024982324, cv 0.9878367476762702
time_slice 70: train 0.15355184255258614, cv 0.6065469989549293
time_slice 82: train 0.22755916683799746, cv 0.885335221903017
time_slice 94: train 1.4268547697263527, cv 1.4209160617885193
time_slice 106: train 0.44834708504356263, cv 4.707828025762984
time_slice 118: train 0.30370321966380787, cv 0.5477692219850009
time_slice 130: train 0.18306401418356957, cv 0.21053056290387717
time_slice 142: train 0.28064372102462687, cv 1.0565102046674615


In [143]:
training_ts.as_matrix(columns=['gap']).ravel()

array([ 131.,  163.,  102.,  147.,   19.,  121.,   63.,  137.,   45.,
         34.,   13.,   28.,   21.,   46.,  186.,   11.])

In [44]:
# y baseline by all 1
y_train = cv_time['gap'].as_matrix() # transform to np.array

MAPE_scorer(y_train, [1 for i in range(0, len(y_train))])

error, num: 824.921539981 2376


0.34718920032871425

In [64]:
X_training = training.as_matrix(columns=['time_slice','order_count','succeed_count',
                                  'is_workday',
                                  'order_30min','succeed_30min','gap_30min',
                                  'order_20min', 'succeed_20min', 'gap_20min', 
                                  'order_10min', 'succeed_10min', 'gap_10min'])
y_training = training.as_matrix(columns=['gap']).ravel()

X_cv = cv.as_matrix(columns=['time_slice','order_count','succeed_count',
                                  'is_workday',
                                  'order_30min','succeed_30min','gap_30min',
                                  'order_20min', 'succeed_20min', 'gap_20min', 
                                  'order_10min', 'succeed_10min', 'gap_10min'])
y_cv = cv.as_matrix(columns=['gap']).ravel()

X_cv_time = cv_time.as_matrix(columns=['time_slice','order_count','succeed_count',
                                  'is_workday',
                                  'order_30min','succeed_30min','gap_30min',
                                  'order_20min', 'succeed_20min', 'gap_20min', 
                                  'order_10min', 'succeed_10min', 'gap_10min'])
y_cv_time = cv_time.as_matrix(columns=['gap']).ravel()

In [58]:
X

array([[   1.,   88.,   85., ...,   84.,   82.,    2.],
       [   2.,   97.,   90., ...,   88.,   85.,    3.],
       [   3.,   75.,   70., ...,   97.,   90.,    7.],
       ..., 
       [ 142.,   64.,   63., ...,   63.,   61.,    2.],
       [ 143.,   61.,   58., ...,   64.,   63.,    1.],
       [ 144.,   51.,   47., ...,   61.,   58.,    3.]])

In [94]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation

clf = RandomForestRegressor(n_estimators=10)
clf = clf.fit(X_training, y_training)

print(clf.score(X_training, y_training))
print(clf.score(X_cv, y_cv))
print(clf.score(X_cv_time, y_cv_time))

0.926847529234
0.835285154662
0.50253375695


In [88]:
from sklearn.svm import SVR

clf = SVR(C=1.0, epsilon=0.2)
clf = clf.fit(X_training, y_training)

print(clf.score(X_training, y_training))
print(clf.score(X_cv, y_cv))
print(clf.score(X_cv_time, y_cv_time))

0.0297957532569
-0.0205079220299
-0.0923028979494


In [95]:
print(clf.predict(X_cv_time))
print(y_cv_time)

[  2.7   2.8   9.3   4.3   5.4   8.1   6.2   5.6   2.6   2.4   4.1   6.
   4.1   3.3   4.    6.2   3.5   3.2   2.7   3.6   9.2   3.    5.1   8.5
   7.7   8.2   2.6   8.8   5.7   4.    3.6   6.2  20.3   3.9   6.3   3.4]
[  3.   0.   8.   4.   4.  10.   9.   7.   4.   6.   6.   6.   3.   2.   1.
   6.   0.   1.   2.   0.  17.   1.   5.  13.   9.  11.   1.   5.   4.   2.
   2.   5.  12.   4.   7.   4.]


In [100]:

print(MAPE_scorer(y_cv, clf.predict(X_cv)))
print(MAPE_scorer(y_cv_time, clf.predict(X_cv_time)/2))

error, num: 282.053309219 576
0.489675884061
error, num: 15.7112891928 36
0.436424699801


In [104]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(training.as_matrix(columns=['time_slice']))
enc.n_values_

array([145])

In [105]:
enc.feature_indices_

array([  0, 145])